## Weaviate Quickstart - Python code

This is an example Python (Jupyter) notebook for the [Weaviate Quickstart tutorial](https://weaviate.io/developers/weaviate/quickstart).

<a target="_blank" href="https://colab.research.google.com/github/weaviate/weaviate-examples/blob/main/quickstart_code/quickstart_code.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

*Last updated: 2 Feb 2023*

In [1]:
# Load OpenAI token - replace with script to load your token, e.g. from your environment variables, via:
# import os
# api_tkn = os.getenv('OPENAI_APIKEY')

with open('../../keys/openai_key.txt', 'r') as f:
    api_tkn = f.read().strip()

# Import libraries
import weaviate
import json

# Instantiate Weaviate client
client = weaviate.Client(
    url="https://some-endpoint.weaviate.network",  # Replace this with your WCS or Docker endpoint (use HTTP, not HTTPS for local addresses)
    additional_headers={
        "X-OpenAI-Api-Key": api_tkn  # Or "X-Cohere-Api-Key" or "X-HuggingFace-Api-Key" 
    }
)

In [2]:
# Delete existing schema (if necessary - THIS WILL ALSO DELETE ALL OF YOUR DATA)
client.schema.delete_all()

# Fetch & inspect schema (should be empty)
schema = client.schema.get()
print(json.dumps(schema, indent=4))

{
    "classes": []
}


In [3]:
# ===== add schema ===== 
class_obj = {
    "class": "Question",
    "vectorizer": "text2vec-openai"  # Or "text2vec-cohere" or "text2vec-huggingface"
}

client.schema.create_class(class_obj)

In [4]:
# ===== import data ===== 
# Load data from the data.json file
with open('jeopardy_tiny.json', 'r') as f:
    data = json.load(f)

# Configure a batch process
with client.batch as batch:
    batch.batch_size=100
    # Batch import all Questions
    for i, d in enumerate(data):
        print(f"importing question: {i+1}")

        properties = {
            "answer": d["Answer"],
            "question": d["Question"],
            "category": d["Category"],
        }

        client.batch.add_data_object(properties, "Question")

importing question: 1
importing question: 2
importing question: 3
importing question: 4
importing question: 5
importing question: 6
importing question: 7
importing question: 8
importing question: 9
importing question: 10


In [5]:
client.data_object.get()

{'deprecations': None,
 'objects': [{'class': 'Question',
   'creationTimeUnix': 1675299167735,
   'id': '007f2d5f-9522-4596-913a-b3c669df1b89',
   'lastUpdateTimeUnix': 1675299167735,
   'properties': {'answer': 'species',
    'category': 'SCIENCE',
    'question': "2000 news: the Gunnison sage grouse isn't just another northern sage grouse, but a new one of this classification"},
   'vectorWeights': None},
  {'class': 'Question',
   'creationTimeUnix': 1675299167736,
   'id': '4528b6bb-f091-4403-8359-04b18f657305',
   'lastUpdateTimeUnix': 1675299167736,
   'properties': {'answer': 'the nose or snout',
    'category': 'ANIMALS',
    'question': 'The gavial looks very much like a crocodile except for this bodily feature'},
   'vectorWeights': None},
  {'class': 'Question',
   'creationTimeUnix': 1675299167736,
   'id': '4a39f51a-0bb2-4a8e-b55a-c39bd4227e60',
   'lastUpdateTimeUnix': 1675299167736,
   'properties': {'answer': 'Antelope',
    'category': 'ANIMALS',
    'question': 'Weig

In [6]:
nearText = {"concepts": ["biology"]}

result = (
    client.query
    .get("Question", ["question", "answer", "category"])
    .with_near_text(nearText)
    .with_limit(2)
    .do()
)

print(json.dumps(result, indent=4))

{
    "data": {
        "Get": {
            "Question": [
                {
                    "answer": "DNA",
                    "category": "SCIENCE",
                    "question": "In 1953 Watson & Crick built a model of the molecular structure of this, the gene-carrying substance"
                },
                {
                    "answer": "species",
                    "category": "SCIENCE",
                    "question": "2000 news: the Gunnison sage grouse isn't just another northern sage grouse, but a new one of this classification"
                }
            ]
        }
    }
}
